In [382]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.preprocessing import LabelEncoder

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import warnings

warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [383]:
train_df = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv")

train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [384]:
train_df.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

In [385]:
test_df.isnull().sum()

id                                           0
Name                                         0
Gender                                       0
Age                                          0
City                                         0
Working Professional or Student              0
Profession                               24632
Academic Pressure                        75033
Work Pressure                            18778
CGPA                                     75034
Study Satisfaction                       75033
Job Satisfaction                         18774
Sleep Duration                               0
Dietary Habits                               5
Degree                                       2
Have you ever had suicidal thoughts ?        0
Work/Study Hours                             0
Financial Stress                             0
Family History of Mental Illness             0
dtype: int64

In [386]:
for col in train_df.select_dtypes('object').columns:
    train_df[col] = train_df[col].fillna('blank')

for col in test_df.select_dtypes('object').columns:
    test_df[col] = test_df[col].fillna('blank')

In [387]:
for col in [cols for cols in train_df.select_dtypes('number').columns]:
    train_df[col]=train_df[col].fillna(0)

for col in [cols for cols in test_df.select_dtypes('number').columns]:
    test_df[col]=test_df[col].fillna(0)

In [388]:
map_gender = {'Male':0, 'Female':1}

train_df['Gender'] = train_df['Gender'].map(map_gender)
test_df['Gender'] = test_df['Gender'].map(map_gender)

map_WPS = {'Working Professional':0, 'Student':1}

train_df['Working Professional or Student'] = train_df['Working Professional or Student'].map(map_WPS)
test_df['Working Professional or Student'] = test_df['Working Professional or Student'].map(map_WPS)

map_thoughts = {'Yes':0, 'No':1}

train_df['Have you ever had suicidal thoughts ?'] = train_df['Have you ever had suicidal thoughts ?'].map(map_thoughts)
test_df['Have you ever had suicidal thoughts ?'] = test_df['Have you ever had suicidal thoughts ?'].map(map_thoughts)

map_family = {'Yes':0, 'No':1}

train_df['Family History of Mental Illness'] = train_df['Family History of Mental Illness'].map(map_family)
test_df['Family History of Mental Illness'] = test_df['Family History of Mental Illness'].map(map_family)

In [389]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,1,49.0,Ludhiana,0,Chef,0.0,5.0,0.00,0.0,2.0,More than 8 hours,Healthy,BHM,1,1.0,2.0,1,0
1,1,Vivan,0,26.0,Varanasi,0,Teacher,0.0,4.0,0.00,0.0,3.0,Less than 5 hours,Unhealthy,LLB,0,7.0,3.0,1,1
2,2,Yuvraj,0,33.0,Visakhapatnam,1,blank,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,0,3.0,1.0,1,1
3,3,Yuvraj,0,22.0,Mumbai,0,Teacher,0.0,5.0,0.00,0.0,1.0,Less than 5 hours,Moderate,BBA,0,10.0,1.0,0,1
4,4,Rhea,1,30.0,Kanpur,0,Business Analyst,0.0,1.0,0.00,0.0,1.0,5-6 hours,Unhealthy,BBA,0,9.0,4.0,0,0


In [390]:
test_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,0,53.0,Visakhapatnam,0,Judge,0.0,2.0,0.00,0.0,5.0,Less than 5 hours,Moderate,LLB,1,9.0,3.0,0
1,140701,Sanya,1,58.0,Kolkata,0,Educational Consultant,0.0,2.0,0.00,0.0,4.0,Less than 5 hours,Moderate,B.Ed,1,6.0,4.0,1
2,140702,Yash,0,53.0,Jaipur,0,Teacher,0.0,4.0,0.00,0.0,1.0,7-8 hours,Moderate,B.Arch,0,12.0,4.0,1
3,140703,Nalini,1,23.0,Rajkot,1,blank,5.0,0.0,6.84,1.0,0.0,More than 8 hours,Moderate,BSc,0,10.0,4.0,1
4,140704,Shaurya,0,47.0,Kalyan,0,Teacher,0.0,5.0,0.00,0.0,5.0,7-8 hours,Moderate,BCA,0,3.0,4.0,1


In [391]:
le = LabelEncoder()

le = le.fit(train_df['City'])

train_df['City'] = le.transform(train_df['City'])

for label in test_df['City']:
    if label not in le.classes_:
        le.classes_ = np.append(le.classes_,label)

test_df['City'] = le.transform(test_df['City'])

le = le.fit(train_df['Profession'])

train_df['Profession'] = le.transform(train_df['Profession'])

for label in test_df['Profession']:
    if label not in le.classes_:
        le.classes_ = np.append(le.classes_,label)

test_df['Profession'] = le.transform(test_df['Profession'])


le = le.fit(train_df['Degree'])

train_df['Degree'] = le.transform(train_df['Degree'])

for label in test_df['Degree']:
    if label not in le.classes_:
        le.classes_ = np.append(le.classes_,label)

test_df['Degree'] = le.transform(test_df['Degree'])

In [392]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,1,49.0,50,0,10,0.0,5.0,0.00,0.0,2.0,More than 8 hours,Healthy,33,1,1.0,2.0,1,0
1,1,Vivan,0,26.0,93,0,55,0.0,4.0,0.00,0.0,3.0,Less than 5 hours,Unhealthy,63,0,7.0,3.0,1,1
2,2,Yuvraj,0,33.0,97,1,64,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,21,0,3.0,1.0,1,1
3,3,Yuvraj,0,22.0,64,0,55,0.0,5.0,0.00,0.0,1.0,Less than 5 hours,Moderate,28,0,10.0,1.0,0,1
4,4,Rhea,1,30.0,37,0,9,0.0,1.0,0.00,0.0,1.0,5-6 hours,Unhealthy,28,0,9.0,4.0,0,0


In [393]:
test_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,0,53.0,97,0,31,0.0,2.0,0.00,0.0,5.0,Less than 5 hours,Moderate,63,1,9.0,3.0,0
1,140701,Sanya,1,58.0,44,0,21,0.0,2.0,0.00,0.0,4.0,Less than 5 hours,Moderate,20,1,6.0,4.0,1
2,140702,Yash,0,53.0,33,0,55,0.0,4.0,0.00,0.0,1.0,7-8 hours,Moderate,17,0,12.0,4.0,1
3,140703,Nalini,1,23.0,78,1,64,5.0,0.0,6.84,1.0,0.0,More than 8 hours,Moderate,36,0,10.0,4.0,1
4,140704,Shaurya,0,47.0,36,0,55,0.0,5.0,0.00,0.0,5.0,7-8 hours,Moderate,29,0,3.0,4.0,1


In [394]:
train_df['Sleep Duration'].unique()

array(['More than 8 hours', 'Less than 5 hours', '5-6 hours', '7-8 hours',
       'Sleep_Duration', '1-2 hours', '6-8 hours', '4-6 hours',
       '6-7 hours', '10-11 hours', '8-9 hours', '40-45 hours',
       '9-11 hours', '2-3 hours', '3-4 hours', 'Moderate', '55-66 hours',
       '4-5 hours', '9-6 hours', '1-3 hours', 'Indore', '45', '1-6 hours',
       '35-36 hours', '8 hours', 'No', '10-6 hours', 'than 5 hours',
       '49 hours', 'Unhealthy', 'Work_Study_Hours', '3-6 hours',
       '45-48 hours', '9-5', 'Pune', '9-5 hours'], dtype=object)

In [395]:
map_sleep = {
    'More than 8 hours': 8.5, #mapping with meadian
    'Less than 5 hours': 4.5,
    '5-6 hours': 5.5,
    '7-8 hours': 7.5,
    '1-2 hours': 1.5,
    '6-8 hours': 7,
    '4-6 hours': 5,
    '6-7 hours': 6.5,
    '10-11 hours': 10.5,
    '8-9 hours': 8.5,
    '40-45 hours': 6, # Assume that the surveyor wrote down a week of sleep
    '9-11 hours': 10,
    '2-3 hours': 2.5,
    '3-4 hours': 3.5,
    'Moderate': 7,
    '55-66 hours': 8.6,
    '4-5 hours': 4.5,
    '9-6 hours': 7.5,
    '1-3 hours': 2,
    '45': 6.4,
    '1-6 hours': 3.5,
    '35-36 hours': 5,
    '8 hours': 8,
    '10-6 hours': 8,
    'than 5 hours': 5.5, # Assume "More" than 5 hours
    '49 hours': 7,
    'Unhealthy': 1.5, # Min value
    '3-6 hours': 4.5,
    '45-48 hours': 6.6,
    '9-5': 7,
    '9-5 hours': 7
}

train_df['Sleep Duration'] = train_df['Sleep Duration'].map(map_sleep)
test_df['Sleep Duration'] = test_df['Sleep Duration'].map(map_sleep)

In [396]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,1,49.0,50,0,10,0.0,5.0,0.00,0.0,2.0,8.5,Healthy,33,1,1.0,2.0,1,0
1,1,Vivan,0,26.0,93,0,55,0.0,4.0,0.00,0.0,3.0,4.5,Unhealthy,63,0,7.0,3.0,1,1
2,2,Yuvraj,0,33.0,97,1,64,5.0,0.0,8.97,2.0,0.0,5.5,Healthy,21,0,3.0,1.0,1,1
3,3,Yuvraj,0,22.0,64,0,55,0.0,5.0,0.00,0.0,1.0,4.5,Moderate,28,0,10.0,1.0,0,1
4,4,Rhea,1,30.0,37,0,9,0.0,1.0,0.00,0.0,1.0,5.5,Unhealthy,28,0,9.0,4.0,0,0


In [397]:
test_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,0,53.0,97,0,31,0.0,2.0,0.00,0.0,5.0,4.5,Moderate,63,1,9.0,3.0,0
1,140701,Sanya,1,58.0,44,0,21,0.0,2.0,0.00,0.0,4.0,4.5,Moderate,20,1,6.0,4.0,1
2,140702,Yash,0,53.0,33,0,55,0.0,4.0,0.00,0.0,1.0,7.5,Moderate,17,0,12.0,4.0,1
3,140703,Nalini,1,23.0,78,1,64,5.0,0.0,6.84,1.0,0.0,8.5,Moderate,36,0,10.0,4.0,1
4,140704,Shaurya,0,47.0,36,0,55,0.0,5.0,0.00,0.0,5.0,7.5,Moderate,29,0,3.0,4.0,1


In [398]:
train_df['Sleep Duration'] = train_df['Sleep Duration'].fillna(train_df['Sleep Duration'].median())
test_df['Sleep Duration'] = test_df['Sleep Duration'].fillna(train_df['Sleep Duration'].median())

In [399]:
train_df.isnull().sum()

id                                       0
Name                                     0
Gender                                   0
Age                                      0
City                                     0
Working Professional or Student          0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64

In [400]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,1,49.0,50,0,10,0.0,5.0,0.00,0.0,2.0,8.5,Healthy,33,1,1.0,2.0,1,0
1,1,Vivan,0,26.0,93,0,55,0.0,4.0,0.00,0.0,3.0,4.5,Unhealthy,63,0,7.0,3.0,1,1
2,2,Yuvraj,0,33.0,97,1,64,5.0,0.0,8.97,2.0,0.0,5.5,Healthy,21,0,3.0,1.0,1,1
3,3,Yuvraj,0,22.0,64,0,55,0.0,5.0,0.00,0.0,1.0,4.5,Moderate,28,0,10.0,1.0,0,1
4,4,Rhea,1,30.0,37,0,9,0.0,1.0,0.00,0.0,1.0,5.5,Unhealthy,28,0,9.0,4.0,0,0


In [401]:
train_df['Dietary Habits'].unique()

array(['Healthy', 'Unhealthy', 'Moderate', 'Yes', 'Pratham', 'BSc',
       'Gender', '3', 'More Healthy', 'Less than Healthy', 'Mihir', '1.0',
       'Hormonal', 'Electrician', 'blank', 'No Healthy', 'Less Healthy',
       'M.Tech', 'Vegas', 'No', 'Male', 'Indoor', 'Class 12', '2'],
      dtype=object)

In [402]:
map_habits = {
    'More Healthy': 4,
    'Healthy': 3,
    'Moderate': 2,
    'Less than Healthy': 1,
    'Less Healthy': 1,
    'No Healthy': 0,
    'Unhealthy': 0
}

train_df['Dietary Habits'] = train_df['Dietary Habits'].map(map_habits)
test_df['Dietary Habits'] = test_df['Dietary Habits'].map(map_habits)

In [403]:
train_df['Dietary Habits'] = train_df['Dietary Habits'].fillna(train_df['Dietary Habits'].median())
test_df['Dietary Habits'] = test_df['Dietary Habits'].fillna(train_df['Dietary Habits'].median())

In [404]:
train_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,1,49.0,50,0,10,0.0,5.0,0.00,0.0,2.0,8.5,3.0,33,1,1.0,2.0,1,0
1,1,Vivan,0,26.0,93,0,55,0.0,4.0,0.00,0.0,3.0,4.5,0.0,63,0,7.0,3.0,1,1
2,2,Yuvraj,0,33.0,97,1,64,5.0,0.0,8.97,2.0,0.0,5.5,3.0,21,0,3.0,1.0,1,1
3,3,Yuvraj,0,22.0,64,0,55,0.0,5.0,0.00,0.0,1.0,4.5,2.0,28,0,10.0,1.0,0,1
4,4,Rhea,1,30.0,37,0,9,0.0,1.0,0.00,0.0,1.0,5.5,0.0,28,0,9.0,4.0,0,0


In [405]:
test_df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,0,53.0,97,0,31,0.0,2.0,0.00,0.0,5.0,4.5,2.0,63,1,9.0,3.0,0
1,140701,Sanya,1,58.0,44,0,21,0.0,2.0,0.00,0.0,4.0,4.5,2.0,20,1,6.0,4.0,1
2,140702,Yash,0,53.0,33,0,55,0.0,4.0,0.00,0.0,1.0,7.5,2.0,17,0,12.0,4.0,1
3,140703,Nalini,1,23.0,78,1,64,5.0,0.0,6.84,1.0,0.0,8.5,2.0,36,0,10.0,4.0,1
4,140704,Shaurya,0,47.0,36,0,55,0.0,5.0,0.00,0.0,5.0,7.5,2.0,29,0,3.0,4.0,1


In [406]:
train_df.drop(['id', 'Name'],axis=1,inplace=True)

In [407]:
X = train_df.drop('Depression', axis=1).copy()
Y = train_df['Depression'].copy()

In [408]:
X.head()

,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,1,49.0,50,0,10,0.0,5.0,0.00,0.0,2.0,8.5,3.0,33,1,1.0,2.0,1
1,0,26.0,93,0,55,0.0,4.0,0.00,0.0,3.0,4.5,0.0,63,0,7.0,3.0,1
2,0,33.0,97,1,64,5.0,0.0,8.97,2.0,0.0,5.5,3.0,21,0,3.0,1.0,1
3,0,22.0,64,0,55,0.0,5.0,0.00,0.0,1.0,4.5,2.0,28,0,10.0,1.0,0
4,1,30.0,37,0,9,0.0,1.0,0.00,0.0,1.0,5.5,0.0,28,0,9.0,4.0,0


In [409]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [410]:
clf_xgb = xgb.XGBClassifier(object='binary:logistic', missing=None)
clf_xgb.fit(
    X_train,
    Y_train,
    verbose=True,
    early_stopping_rounds=10,
    eval_metric='aucpr',
    eval_set=[(X_test,Y_test)]
)

[0]	validation_0-aucpr:0.85742
[1]	validation_0-aucpr:0.87555
[2]	validation_0-aucpr:0.88264
[3]	validation_0-aucpr:0.88484
[4]	validation_0-aucpr:0.88688
[5]	validation_0-aucpr:0.89023
[6]	validation_0-aucpr:0.89222
[7]	validation_0-aucpr:0.89339
[8]	validation_0-aucpr:0.89491
[9]	validation_0-aucpr:0.89620
[10]	validation_0-aucpr:0.89897
[11]	validation_0-aucpr:0.89968
[12]	validation_0-aucpr:0.90156
[13]	validation_0-aucpr:0.90234
[14]	validation_0-aucpr:0.90264
[15]	validation_0-aucpr:0.90362
[16]	validation_0-aucpr:0.90371
[17]	validation_0-aucpr:0.90395
[18]	validation_0-aucpr:0.90381
[19]	validation_0-aucpr:0.90411
[20]	validation_0-aucpr:0.90419
[21]	validation_0-aucpr:0.90453
[22]	validation_0-aucpr:0.90474
[23]	validation_0-aucpr:0.90479
[24]	validation_0-aucpr:0.90483
[25]	validation_0-aucpr:0.90481
[26]	validation_0-aucpr:0.90508
[27]	validation_0-aucpr:0.90534
[28]	validation_0-aucpr:0.90525
[29]	validation_0-aucpr:0.90532
[30]	validation_0-aucpr:0.90524
[31]	validation_0-

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=None, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, object='binary:logistic', ...)

In [411]:
param_grid={
    'max_depth': [3,4,5],
    'learning_rate': [1, 0.5, 0.1, 0.01, 0.05],
    'gamma': [0, 0.25, 1.0],
    'reg_lambda': [0, 1.0, 10.0, 20, 100],
    'scale_pos_weght': [1, 3, 5]
}

optimal_params = GridSearchCV(
    estimator=xgb.XGBClassifier(
        object='binary:logistic',
        subsample=0.9,
        colsample_bytree=0.5
    ),
    param_grid=param_grid,
    scoring='roc_auc',
    verbose=0,
    n_jobs=10,
    cv=3
)

In [412]:
clf_xgb = xgb.XGBClassifier(object='binary:logistic',
                            gamma=0.25,
                            learning_rate=0.1,
                            max_depth=4,
                            reg_lambda=10,
                            scale_pos_weght=3,
                            subsample=0.9,
                            colsample_bytree=0.5
                           )

clf_xgb.fit(
    X_train,
    Y_train,
    verbose=True,
    early_stopping_rounds=10,
    eval_metric='aucpr',
    eval_set=[(X_test,Y_test)]
)

[0]	validation_0-aucpr:0.65722
[1]	validation_0-aucpr:0.74662
[2]	validation_0-aucpr:0.74408
[3]	validation_0-aucpr:0.74256
[4]	validation_0-aucpr:0.78417
[5]	validation_0-aucpr:0.83199
[6]	validation_0-aucpr:0.82636
[7]	validation_0-aucpr:0.83816
[8]	validation_0-aucpr:0.84647
[9]	validation_0-aucpr:0.85158
[10]	validation_0-aucpr:0.84984
[11]	validation_0-aucpr:0.85716
[12]	validation_0-aucpr:0.85535
[13]	validation_0-aucpr:0.86033
[14]	validation_0-aucpr:0.86220
[15]	validation_0-aucpr:0.86477
[16]	validation_0-aucpr:0.86516
[17]	validation_0-aucpr:0.86807
[18]	validation_0-aucpr:0.87363
[19]	validation_0-aucpr:0.87423
[20]	validation_0-aucpr:0.87699
[21]	validation_0-aucpr:0.88050
[22]	validation_0-aucpr:0.88490
[23]	validation_0-aucpr:0.88782
[24]	validation_0-aucpr:0.88770
[25]	validation_0-aucpr:0.88770
[26]	validation_0-aucpr:0.88903
[27]	validation_0-aucpr:0.89021
[28]	validation_0-aucpr:0.88988
[29]	validation_0-aucpr:0.89079
[30]	validation_0-aucpr:0.89189
[31]	validation_0-

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, object='binary:logistic', ...)

In [413]:
test_ids = test_df.pop('id')
X_pred = test_df.drop('Name', axis=1).copy()

In [414]:
test_preds = clf_xgb.predict(X_pred)

In [415]:
submission = pd.DataFrame({'id': test_ids, 'Depression': test_preds})
submission.to_csv('submission.csv', index=False)
submission

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1
